In [ ]:
# Importar funciones de procesamiento de datos
from limpieza_datos import *

# Importar la clase del modelo y funciones de entrenamiento/evaluación
# from GNN import MultiKRWithGCN
from train_and_evaluate import *

from KGs import *

# Importar funciones para modelar el EDA
from edas import *

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Procesar los datos y capturar los mapeos
item_index_old2new, entity_id2index = read_item_index_to_entity_id_file()

In [ ]:
convert_rating(item_index_old2new)
entity_id2index, relation_id2index = convert_kg()

In [ ]:
# Carga o genera edge_index una sola vez antes del entrenamiento
edge_index = load_kg_and_create_edge_index(entity_id2index,relation_id2index)

In [ ]:
# Mapeo de tipos de relaciones a tipos de entidades ajustado a tu descripción
relation_to_entity_types = {
    'book.book.genre': ('book', 'genre'),
    'book.written_work.date_of_first_publication': ('book', 'date'),
    'book.literary_series.author': ('series', 'author'),
    'comic_books.series.publisher': ('comic_series', 'publisher'),
    'book.written_work.author': ('book', 'author'),
    'book.literary_series.works_in_this_series': ('series', 'work'),
    'book.written_work.translation': ('original_work', 'translation'),
    'book.written_work.subject': ('work', 'subject'),
    'book.written_work.literary_series': ('work', 'series'),
    'book.written_work.previous_in_series': ('work', 'previous_work'),
}


# Generar el mapeo de ID de entidad a tipo basado en el archivo kg.txt y el mapeo relation_to_entity_types
entity_to_type = generate_entity_to_type_mapping('./MKR-data/kg.txt', relation_to_entity_types)

# Mapeo de índice de relación a nombre de relación (debes definir este mapeo basado en tu datos)
index_to_relation_name = {
    0: 'book.book.genre',
    1: 'book.written_work.date_of_first_publication',
    2: 'book.literary_series.author',
    3: 'comic_books.series.publisher',
    4: 'book.written_work.author',
    5: 'book.literary_series.works_in_this_series',
    6: 'book.written_work.translation',
    7: 'book.written_work.subject',
    8: 'book.written_work.literary_series',
    9: 'book.written_work.previous_in_series'
}

In [ ]:
kg_file_path = './MKR-data/kg.txt'
output_path = './MKR-data/'

# Llama a la función como antes, omitiendo 'relation_id2index' y 'index_to_relation_name'
# adapt_and_split_kg_data_with_slashes(entity_id2index, relation_to_entity_types, kg_file_path, output_path)

In [ ]:
# Preparar los datos para el entrenamiento y la evaluación
ratings = np.loadtxt('./MKR-data/ratings_final.txt', dtype=np.int32)
train_data, eval_data, test_data = dataset_split(ratings)

# Convertir los conjuntos de datos a DataLoader
train_loader = DataLoader(TrainSet(train_data), batch_size=64, shuffle=True)
val_loader = DataLoader(TrainSet(eval_data), batch_size=64, shuffle=True)
eval_loader = DataLoader(TrainSet(test_data), batch_size=64, shuffle=False)


In [ ]:
print("Training dataset summary:")
summarize_dataset(TrainSet(train_data))

print("\nEvaluation dataset summary:")
summarize_dataset(TrainSet(eval_data))

# Comprobar una muestra de los datos cargados
sample_user, sample_item, sample_target = next(iter(eval_loader))
print("\nSample batch from eval_loader:")
print(f"User tensor: {sample_user}")
print(f"Item tensor: {sample_item}")
print(f"Target tensor: {sample_target}")


## OVERSAMPLING NEGATIVO DE TRIPLETAS

In [ ]:
# Cargar datos del KG
kg_data = np.loadtxt('./MKR-data/kg_final.txt', dtype=int)

# Calcular el número total de entidades en el KG
num_entities = max(np.max(kg_data[:, 0]), np.max(kg_data[:, 2])) + 1

# # Generar ejemplos negativos para los datos del KG
# kg_data_with_negatives = generate_kg_negative_samples(kg_data, num_entities, num_samples=1)

In [ ]:
# # Crear mapeos inversos
# index_to_entity_id = {v: k for k, v in entity_id2index.items()}
# index_to_relation_id = {v: k for k, v in relation_id2index.items()}

In [ ]:
# def translate_triplet(triplet, index_to_entity_id, index_to_relation_id):
#     head, relation, tail, is_positive = triplet
#     head_id = index_to_entity_id.get(head, "Unknown Entity")
#     tail_id = index_to_entity_id.get(tail, "Unknown Entity")
#     relation_id = index_to_relation_id.get(relation, "Unknown Relation")
#     return head_id, relation_id, tail_id, is_positive

# def print_translated_triplets(samples, index_to_entity_id, index_to_relation_id):
#     for sample in samples:
#         translated = translate_triplet(sample, index_to_entity_id, index_to_relation_id)
#         print(f"{translated[0]}\t{translated[1]}\t{translated[2]}\t{'Positive' if translated[3] == 1 else 'Negative'}")

In [ ]:
# # Ejemplos de muestra positiva
# print("Ejemplos de muestra positiva:")
# print_translated_triplets(kg_data_with_negatives[kg_data_with_negatives[:, 3] == 1][:5], index_to_entity_id, index_to_relation_id)

# # Ejemplos de muestra negativa
# print("\nEjemplos de muestra negativa:")
# print_translated_triplets(kg_data_with_negatives[kg_data_with_negatives[:, 3] == 0][:5], index_to_entity_id, index_to_relation_id)


## DATA LOADER

### KG

In [ ]:
# # Supongamos que tienes kg_data con columnas [head, relation_id, tail]
# relation_ids = kg_data[:, 1]  # Extraer los IDs de las relaciones del dataset

# # Crear un mapeo de ID de relación a índice
# unique_relation_ids = np.unique(relation_ids)
# relation_id_to_index = {id: index for index, id in enumerate(unique_relation_ids)}

# # Aplicar el mapeo a tu dataset para convertir IDs de relaciones a índices
# mapped_relation_indices = np.array([relation_id_to_index[id] for id in relation_ids])

# # Reemplazar la columna de ID de relaciones en kg_data por índices mapeados
# kg_data[:, 1] = mapped_relation_indices


In [ ]:
# kg_train_set=KGTrainSet(kg_data_with_negatives)
kg_train_set=KGTrainSet(kg_data)
kg_train_loader = DataLoader(kg_train_set, batch_size=64, shuffle=True)

### REC

In [ ]:
# Preparar los datos para el entrenamiento y la evaluación de recomendaciones
ratings = np.loadtxt('./MKR-data/ratings_final.txt', dtype=np.int32)
train_data, eval_data, test_data = dataset_split(ratings)

# Convertir los conjuntos de datos a DataLoader para las recomendaciones
rec_train_loader = DataLoader(TrainSet(train_data), batch_size=64, shuffle=True)
rec_val_loader = DataLoader(TrainSet(eval_data), batch_size=64, shuffle=True)
rec_eval_loader = DataLoader(TrainSet(test_data), batch_size=64, shuffle=False)


In [ ]:
# Inicializar el modelo MultiKR
user_num = len(np.unique(ratings[:, 0]))  # Número de usuarios únicos
item_num = len(item_index_old2new)  # Número de ítems únicos
entity_num = len(entity_id2index)  # Número de entidades únicas
relation_num = len(np.unique(kg_data[:, 1])) # Número de relaciones únicas

In [ ]:
# Datos para el entrenamiento de KG
print("KG Training Dataset:")
print(f"- Total triplets (with negatives): {len(kg_train_set)}")
print(f"  (Cada tripleta incluye una entidad de cabeza, una relación, una entidad de cola, y una etiqueta indicando si es un ejemplo positivo o negativo)")

# Datos para el entrenamiento de recomendaciones
print("\nRecommendation Training Dataset:")
print(f"- Total ratings (train): {len(train_data)}")
print(f"- Total ratings (validation): {len(eval_data)}")
print(f"- Total ratings (test): {len(test_data)}")
print("  (Cada rating incluye un usuario, un ítem, y una etiqueta indicando la interacción)")

# Información general sobre el modelo MultiKR
print("\nInformación General del Modelo MultiKR:")
print(f"- Número de usuarios únicos: {user_num}")
print(f"- Número de ítems únicos: {item_num} (ítems mapeados del KG a recomendaciones)")
print(f"- Número de entidades únicas en el KG: {entity_num}")
print(f"- Número de relaciones únicas en el KG: {relation_num}")
print("  (Esta configuración se utiliza para inicializar el modelo MultiKR, que integra datos de KG y recomendaciones)")


## KG MODEL

In [ ]:
# Supongamos los siguientes parámetros
n_layer = 1       # Número de capas en la red
embed_dim = 50    # Dimensión de los embeddings
hidden_layers = [64]  # Capas ocultas
dropouts = [0.5]  # Dropout para cada capa
output_rec = 1    # Salida para la recomendación
activation_fn = 'leaky_relu'  # Función de activación

kg_hyperparams={'batch_size': 256, 'epochs': 500, 'learning_rate': 0.001, 'optimizer': 'adam'}

# Ruta al conjunto de datos KG
kg_dataset_path = "/home/victor/Escritorio/TFM/git/TFM/Victor/Codigo/GRAFOS/MKR-data/"


entity_embedding_path = "/home/victor/Escritorio/TFM/git/TFM/Victor/Codigo/GRAFOS/MKR-data/embeddings/transe/ent_embedding.tsv"
relation_embedding_path = "/home/victor/Escritorio/TFM/git/TFM/Victor/Codigo/GRAFOS/MKR-data/embeddings/transe/rel_embedding.tsv"

In [ ]:
# Instanciar AdvancedMultiKRWithPykg2vec
model = AdvancedMultiKRWithPykg2vec(
    user_num=user_num, 
    item_num=item_num, 
    entity_num=entity_num,
    relation_num=relation_num,
    n_layer=n_layer, 
    embed_dim=embed_dim,
    hidden_layers=hidden_layers, 
    dropouts=dropouts, 
    output_rec=output_rec,
    activation_fn=activation_fn, 
    kg_model_name='transe', 
    kg_hyperparams=kg_hyperparams,
    kg_dataset_path=kg_dataset_path,
    kg_trained=True,
    entity_embedding_path = "/home/victor/Escritorio/TFM/git/TFM/Victor/Codigo/GRAFOS/MKR-data/embeddings/transe/ent_embedding.tsv",
    relation_embedding_path = "/home/victor/Escritorio/TFM/git/TFM/Victor/Codigo/GRAFOS/MKR-data/embeddings/transe/rel_embedding.tsv",
    freeze=False

)

In [ ]:
# # Ahora, entrena el modelo KG
# model.train_kg_model()

In [ ]:
# Asegúrate de inicializar el optimizador y la función de pérdida adecuados
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_function = torch.nn.BCEWithLogitsLoss()

# Ejecutar entrenamiento y evaluación
train_and_evaluate_rec(model, rec_train_loader, rec_val_loader, optimizer, loss_function, epochs=1)

In [ ]:
# Obteniendo variables y posibles valores para KG y Recomendación
variables_kg, variables_rec = define_variables_for_KG_and_rec()

# Combinar todas las variables y posibles valores en estructuras separadas
# all_variables = list(variables_kg.keys()) + list(variables_rec.keys())
# all_possible_values = list(variables_kg.values()) + list(variables_rec.values())

combined_variables_and_values = {**variables_kg, **variables_rec}


In [ ]:
type(variables_kg)

In [ ]:
# decode_solution_kg(variables_kg), decode_solution_rec(variables_rec)

In [ ]:
# Convertir combined_variables_and_values en la estructura esperada por EBNA
possible_values_numeric = {i: combined_variables_and_values[var] for i, var in enumerate(combined_variables_and_values)}
frequency_numeric = {i: [1/len(possible_values_numeric[i])] * len(possible_values_numeric[i]) for i in possible_values_numeric}

In [ ]:
len(possible_values_numeric)

In [ ]:
# # Convertir posibles valores en una lista de listas para cada variable
# possible_values_list = all_possible_values

# # Inicializar la frecuencia uniforme para cada conjunto de valores posibles
# frequency_list = [[1.0 / len(values) for _ in values] for values in possible_values_list]


In [ ]:
from EDAspy.optimization import EBNA

# Inicializar EBNA con el espacio de soluciones y frecuencias definidas
ebna = EBNA(
    size_gen=20,
    max_iter=20,
    dead_iter=5,
    n_variables=len(possible_values_numeric),
    alpha=0.5,
    possible_values=possible_values_numeric,
    frequency=frequency_numeric
)

In [ ]:
# Definir la función de envoltura para EBNA, pasando los parámetros necesarios
multiKR_cost_wrapper_with_params = lambda solution: multiKR_cost_wrapper_eda(solution)

In [ ]:
# Ejecutar EBNA
ebna_result = ebna.minimize(multiKR_cost_wrapper_with_params)

In [ ]:
# # Simular una entrada como la que esperaríamos de EBNA
# solution_array = np.array([64, 0.001, 128, '64_128', 0.5, 1, 10])

# # Llamar a decode_solution con esta entrada simulada
# decoded_solution = decode_solution(solution_array)
# print("Decoded solution:", decoded_solution)


In [ ]:
# wrapper_result = multiKR_cost_wrapper(decoded_solution)

In [ ]:
# #  Entrenamiento de prueba

# print("Wrapper result:", wrapper_result)
